### Overview

This is seventh in a series tutorials that illustrate how to download the <a href="https://aws.amazon.com/public-data-sets/irs-990/">IRS 990 e-file data</a>. In a series of <a href="http://social-metrics.org/sox/">previous tutorials</a> I have shown how to download data from IRS 990, 990PF, and 990EZ filers. In this tutorial we are going to generate a codebook or 'data dictionary' that can help us make sense of the variables we have downloaded.

### Load Packages

In [1]:
import sys
import time
import json

In [2]:
import numpy as np
import pandas as pd

from pandas import DataFrame
from pandas import Series

In [3]:
#http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 250)

### The need for a codebook

As you can tell from looking at the following sample filing, the data by themselves are difficult to interpret and difficult to match to the various sections of the paper 990 documents: https://s3.amazonaws.com/irs-form-990/201543109349200219_public.xml

To get the variable descriptions you need to dig through the 'schemas', such as that here for 2015: https://www.irs.gov/e-file-providers/990-990-ez-990-pf-ty2015-v30-schema-business-rules-and-release-memo 
  
The IRS makes it difficult to find the <code>*.xsd</code> stylesheets that can be used to make sense of the data. You can visit here -- https://www.irs.gov/downloads/irs-schema/ -- sort the files by Name, and then look for the <code>efile990...</code> zip files. 

Alternatively, you can visit https://www.irs.gov/e-file-providers/current-valid-xml-schemas-and-business-rules-for-exempt-organizations-modernized-e-file -- where you can currently find links to .xsd files for 2014 and 2015. 


For 2015 you would iterate through the links this way:
- https://www.irs.gov/e-file-providers/ty2015-current-valid-xml-schemas-and-business-rules-for-exempt-organizations-modernized-efile-mef then
- https://www.irs.gov/e-file-providers/990-990-ez-990-pf-ty2015-v30-schema-business-rules-and-release-memo then
- https://www.irs.gov/pub/irs-schema/efile990x_2015v3.0.zip


For 2014, the latter will take you here:
- https://www.irs.gov/e-file-providers/ty2014-current-valid-xml-schemas-and-business-rules-for-exempt-organizations-modernized-efile-mef
Then here:
- https://www.irs.gov/e-file-providers/990-990-ez-990-pf-ty2014-v60-schema-business-rules-and-release-memo
Then you will download the "Schemas" zip file:
- https://www.irs.gov/pub/irs-schema/efile990x_2014v6.0_09082015.zip


As a shorthand, here are the direct links to .xsd files for all available years:

- 2015:
    - https://www.irs.gov/pub/irs-schema/efile990x_2015v3.0.zip
    - After unzipping, <code>IRS990.xsd</code> will be found here: &nbsp; <code>/efile990x_2015v3.0/2015v3.0/TEGE/TEGE990/IRS990/IRS990.xsd</code> &nbsp; 
    - Under &nbsp; <code>/TEGE/</code> &nbsp; there are folders for &nbsp;<code>/TEGE990PF/</code> &nbsp; and &nbsp;<code>/TEGE990EZ/</code> &nbsp; and &nbsp;<code>/Common/</code> &nbsp; (which contains &nbsp;<code>*.xsd</code> &nbsp; files for the schedules).


- 2014:
    - https://www.irs.gov/pub/irs-schema/efile990x_2014v6.0_09082015.zip
    - After unzipping, <code>IRS990.xsd</code> will be found here: &nbsp; <code>efile990x_2014v6.0_09082015/2014v6.0/TEGE/TEGE990/IRS990/IRS990.xsd</code> &nbsp;
    - Under &nbsp; <code>/TEGE/</code> &nbsp; there are folders for &nbsp;<code>/TEGE990PF/</code> &nbsp; and &nbsp;<code>/TEGE990EZ/</code> &nbsp; and &nbsp;<code>/Common/</code> &nbsp; (which contains &nbsp;<code>*.xsd</code> &nbsp; files for the schedules).
    


- 2013:
    - https://www.irs.gov/pub/irs-schema/efile990x_2013v4.0.zip
    - After unzipping, <code>IRS990.xsd</code> will be found here: &nbsp; <code>efile990x_2013v4.0/2013v4.0/TEGE/TEGE990/IRS990/IRS990.xsd</code> &nbsp;
    - Under &nbsp; <code>/TEGE/</code> &nbsp; there are folders for &nbsp;<code>/TEGE990PF/</code> &nbsp; and &nbsp;<code>/TEGE990EZ/</code> &nbsp; and &nbsp;<code>/Common/</code> &nbsp; (which contains &nbsp;<code>*.xsd</code> &nbsp; files for the schedules).    


- 2010:
    - https://www.irs.gov/pub/irs-schema/efile990x_2010v3.7.zip  
    - After unzipping, <code>IRS990.xsd</code> will be found here: &nbsp; <code>/efile990x_2010v3.7/2010v3.7/TEGE/TEGE990/IRS990/IRS990.xsd</code> &nbsp; 
    - Under &nbsp; <code>/TEGE/</code> &nbsp; there are folders for &nbsp;<code>/TEGE990PF/</code> &nbsp; and &nbsp;<code>/TEGE990EZ/</code> &nbsp; and &nbsp;<code>/Common/</code> &nbsp; (which contains &nbsp;<code>*.xsd</code> &nbsp; files for the schedules).
    
    
    
    
You should also be aware of the *Nonprofit Open Data Collective*. That project is a more comprehensive effort, particularly their <a href="https://github.com/Nonprofit-Open-Data-Collective/irs-efile-master-concordance-file/blob/master/efiler_master_concordance.csv">master concordance file</a>. My tutorial is mostly intended to highlight the nature of the data and the schemas and serve as an alternative Python version.

### Loop over XSD files, generate relevant dictionary with variable details, and append to PANDAS dataframe
Now we will process the .xsd files. I am assuming you have downloaded the four above *IRS900.xsd* files, put them in a folder called */xsd files/*, and renamed them *IRS990 (2010).xsd*, *IRS990 (2013).xsd*, *IRS990 (2014).xsd*, and *IRS990 (2015).xsd*.

In the following code blocks we will set our working directory and then loop over each of the four xsd files. For each one, we are going to move down to where are the 'elements' are contained. Each element is variable. What we are interested in is the &nbsp; <code>name</code> &nbsp; attribute, the &nbsp; <code>Description</code>, &nbsp; which tells us what the variable means, and the &nbsp; <code>LineNumber</code>, &nbsp; which provides a location for the variable within the 990 form. Together, we have enough information to match the columns in our 990 data to the specific variables we need. Here is an an example of one 'element' in the 2014 xsd file:

			<!-- Total Number Volunteers -->
			<xsd:element name="TotalVolunteersCnt" type="IntegerNNType" minOccurs="0">
				<xsd:annotation>
					<xsd:documentation>
						<Description>Total number volunteers</Description>
						<LineNumber>Part I Line 6</LineNumber>
					</xsd:documentation>
				</xsd:annotation>
			</xsd:element>

Our code below searches over each element and assigns the Description to a variable *description*, the LineNumber to a variable *line_number*, and the year to a variable *year*, and assigns all three variables to a dictionary called *variables*, with the major keys being the variable names. After processing each xsd file, the data are added to a PANDAS dataframe called *variables_df.*

In [ ]:
cd '/Users/gsaxton/Dropbox/990 e-file data/xsd files/'

In [5]:
variables_df = pd.DataFrame()
import xml.etree.ElementTree as ET
for year in ['2010', '2013', '2014', '2015']:
    variables = {}
    tree = ET.parse('IRS990 (%s).xsd' % year)
    root = tree.getroot()
    #print root[3].tag, root[3].attrib
    
    for r in root[3][1].findall("{http://www.w3.org/2001/XMLSchema}element"):
        name = r.attrib['name']
        attrs = r.find('{http://www.w3.org/2001/XMLSchema}annotation/{http://www.w3.org/2001/XMLSchema}documentation')
        description = attrs[0]
        if len(attrs)>1:
            line_number = attrs[1].text
        else:
            line_number = ''
        #print name, description.text, line_number
        
        #if name not in variables:
        #    variables[name] == name
        variables.update({name: {'description': description.text, 'line_number': line_number, 'year': year}})
    
    variables_df = variables_df.append(DataFrame(variables).T) #, ignore_index = True)
    variables_df.index.name = 'variable'

print "# of rows in dataframe:", len(variables_df), '\n'
print "# of rows per year:", variables_df['year'].value_counts(), '\n'
variables_df[:5]  

# of rows in dataframe: 1314 

# of rows per year: 2013    331
2014    331
2015    331
2010    321
Name: year, dtype: int64 



,description,line_number,year
variable,,,
AccountantCompileOrReview,Accountant provide compilation or review?,Part XII Line 2a,2010
AccountsPayableAccruedExpenses,Accounts payable and accrued expenses,Part X Line 17,2010
AccountsReceivable,"Accounts receivable, net",Part X Line 4,2010
ActivitiesConductedPartnership,Activities conducted thru partnership?,Part IV Line 37,2010
Activity2,Second activity group,Part III Line 4b,2010


<br>We see there are 1,314 rows in the database. Above you can see the first five rows and below the last five rows.

In [8]:
variables_df[-5:]

,description,line_number,year
variable,,,
VotingMembersGoverningBodyCnt,Number voting members governing body,Part I Line 3,2015
VotingMembersIndependentCnt,Number independent voting members,Part I Line 4,2015
WebsiteAddressTxt,Web site,J,2015
WhistleblowerPolicyInd,Whistleblower policy?,Part VI Section B Line 13,2015
WrittenPolicyOrProcedureInd,Written policy or procedure?,Part VI Section B Line 16b,2015


<br>We're not quite done, however. To see why, let's reset the index and then sort the dataframe.

In [9]:
variables_df = variables_df.reset_index()
variables_df[:5]

,variable,description,line_number,year
0,AccountantCompileOrReview,Accountant provide compilation or review?,Part XII Line 2a,2010
1,AccountsPayableAccruedExpenses,Accounts payable and accrued expenses,Part X Line 17,2010
2,AccountsReceivable,"Accounts receivable, net",Part X Line 4,2010
3,ActivitiesConductedPartnership,Activities conducted thru partnership?,Part IV Line 37,2010
4,Activity2,Second activity group,Part III Line 4b,2010


In [12]:
variables_df.sort_values(by=['variable', 'year'], ascending=[1,1])[:10]

,variable,description,line_number,year
0,AccountantCompileOrReview,Accountant provide compilation or review?,Part XII Line 2a,2010
321,AccountantCompileOrReviewInd,Accountant provide compilation or review?,Part XII Line 2a,2013
652,AccountantCompileOrReviewInd,Accountant provide compilation or review?,Part XII Line 2a,2014
983,AccountantCompileOrReviewInd,Accountant provide compilation or review?,Part XII Line 2a,2015
322,AccountsPayableAccrExpnssGrp,Accounts payable and accrued expenses,Part X Line 17,2013
653,AccountsPayableAccrExpnssGrp,Accounts payable and accrued expenses,Part X Line 17,2014
984,AccountsPayableAccrExpnssGrp,Accounts payable and accrued expenses,Part X Line 17,2015
1,AccountsPayableAccruedExpenses,Accounts payable and accrued expenses,Part X Line 17,2010
2,AccountsReceivable,"Accounts receivable, net",Part X Line 4,2010
323,AccountsReceivableGrp,"Accounts receivable, net",Part X Line 4,2013


<br>As you can see above, each variable name occurs in more than one row. This is good, because we don't want the IRS to change the variable name every year. A dataframe in this format is also helpful if you are searching by year and then variable name. Yet it becomes problematic if you are searching by variable name. 

So, what we will do is 'collapse' the above dataframe. Specifically, we will search for each row that has an indentical array of values on the first three columns -- *variable*, *description*, and *line_number*. We will then create new columns for the first year the variable-description-line_number array appears, the last year it appears, and all years it appears. 

This is a task PANDAS excels at. It is called a *groupby* operation. I will show you two ways of doing it. The first is with an aggregation *dictionary*. 

In [47]:
# CREATE DICTIONARY OF AGGREGATION OPERATIONS
aggregations = {'year': {'first_year': 'first', 'last_year': 'last', 'all_years': lambda col: ', '.join(col)}}
collapsed_df = variables_df.groupby(['variable', 'description', 'line_number']).agg(aggregations)
#Drop the outermost level from the hierarchical column index:
#http://stackoverflow.com/questions/19078325/naming-returned-columns-in-pandas-aggregate-function
collapsed_df.columns = collapsed_df.columns.droplevel(0)
collapsed_df = collapsed_df.reset_index()
print len(collapsed_df)
collapsed_df[:10]

652


,variable,description,line_number,first_year,last_year,all_years
0,AccountantCompileOrReview,Accountant provide compilation or review?,Part XII Line 2a,2010,2010,2010
1,AccountantCompileOrReviewInd,Accountant provide compilation or review?,Part XII Line 2a,2013,2015,"2013, 2014, 2015"
2,AccountsPayableAccrExpnssGrp,Accounts payable and accrued expenses,Part X Line 17,2013,2015,"2013, 2014, 2015"
3,AccountsPayableAccruedExpenses,Accounts payable and accrued expenses,Part X Line 17,2010,2010,2010
4,AccountsReceivable,"Accounts receivable, net",Part X Line 4,2010,2010,2010
5,AccountsReceivableGrp,"Accounts receivable, net",Part X Line 4,2013,2015,"2013, 2014, 2015"
6,AcctCompileOrReviewBasisGrp,Basis in which the financial statements were compiled or reviewed by independent accountant,Part XII Line 2a,2013,2015,"2013, 2014, 2015"
7,ActivitiesConductedPartnership,Activities conducted thru partnership?,Part IV Line 37,2010,2010,2010
8,ActivitiesConductedPrtshpInd,Activities conducted thru partnership?,Part IV Line 37,2013,2015,"2013, 2014, 2015"
9,Activity2,Second activity group,Part III Line 4b,2010,2010,2010


PANDAS will be <a href="https://stackoverflow.com/questions/44635626/pandas-aggregation-warning-futurewarning-using-a-dict-with-renaming-is-depreca">deprecating the above dictionary approach</a>, so to future-proof the code I will also show you the method that will work in future versions of PANDAS. We need to do this <a href="https://github.com/pandas-dev/pandas/issues/18366#issuecomment-349090402">workaround for 'first' and 'last' aggregation</a> but the end result is the same.

In [50]:
def agg_funcs(x):
    names = {
        'first_year': x['year'].head(1).values[0],
        'last_year':  x['year'].tail(1).values[0],
        #'all_years':  ', '.join(x['year']),
        'all_years':  x['year'].tolist()}
    #THE FOLLOWING SHORTCUT WORKS BUT CHANGES THE ORDER OF THE COLUMNS
    #return pd.Series(names, index = list(names.keys()))
    return pd.Series(names, index=['first_year', 'last_year', 'all_years'])
collapsed_df = variables_df.groupby(['variable', 'description', 'line_number']).apply(agg_funcs)
collapsed_df = collapsed_df.reset_index()
print len(collapsed_df)
collapsed_df[:10]

652


,variable,description,line_number,first_year,last_year,all_years
0,AccountantCompileOrReview,Accountant provide compilation or review?,Part XII Line 2a,2010,2010,[2010]
1,AccountantCompileOrReviewInd,Accountant provide compilation or review?,Part XII Line 2a,2013,2015,"[2013, 2014, 2015]"
2,AccountsPayableAccrExpnssGrp,Accounts payable and accrued expenses,Part X Line 17,2013,2015,"[2013, 2014, 2015]"
3,AccountsPayableAccruedExpenses,Accounts payable and accrued expenses,Part X Line 17,2010,2010,[2010]
4,AccountsReceivable,"Accounts receivable, net",Part X Line 4,2010,2010,[2010]
5,AccountsReceivableGrp,"Accounts receivable, net",Part X Line 4,2013,2015,"[2013, 2014, 2015]"
6,AcctCompileOrReviewBasisGrp,Basis in which the financial statements were compiled or reviewed by independent accountant,Part XII Line 2a,2013,2015,"[2013, 2014, 2015]"
7,ActivitiesConductedPartnership,Activities conducted thru partnership?,Part IV Line 37,2010,2010,[2010]
8,ActivitiesConductedPrtshpInd,Activities conducted thru partnership?,Part IV Line 37,2013,2015,"[2013, 2014, 2015]"
9,Activity2,Second activity group,Part III Line 4b,2010,2010,[2010]


<br>Now we have two variations of our codebook. I would lean toward the latter, but you can take your pick and use whichever suits your particular research purposes. One thing to note above is that there appear to be two types of variables -- those that are used in 2010 and those that are found in the 2013-2015 versions. The complication here is that, if you are searching for, say, the data in *Part XII Line 2a*, you will encounter different variable names according to the year you are working. In effect, we have a sense of the enormity of the problem the *Nonprofit Open Data Collective* is confronting in developing a master variable concordance file.

### Save Dataframes in PANDAS and Excel Formats

In [52]:
variables_df.to_pickle('variable descriptions 2010-2015 990 e-file data.pkl')
variables_df.to_excel('variable descriptions 2010-2015 990 e-file data.xls')
collapsed_df.to_pickle('variable descriptions 2010-2015 990 e-file data (collapsed).pkl')
collapsed_df.to_excel('variable descriptions 2010-2015 990 e-file data (collapsed).xls')

<br>OK, we now have our 990 data and our data codebook. In the next and final tutorial we will employ both to create a dataset with a sample of key 990 variables we are interested in.